In [4]:
import random
import pygame
from main import Game 


try:
    # Inicializar Pygame y el juego
    pygame.init()
    screen_width = 800
    screen_height = 600
    screen = pygame.display.set_mode((screen_width, screen_height))
    clock = pygame.time.Clock()
    game = Game(screen_width, screen_height)

    # Evento para el disparo de los aliens
    ALIENLASER = pygame.USEREVENT + 1
    pygame.time.set_timer(ALIENLASER, 800)  # Configura el temporizador para disparar cada 800 ms

    # Bucle de juego aleatorio
    for _ in range(10000):  # Ejecuta 100 pasos, ajusta según sea necesario
        for event in pygame.event.get():
            if event.type == ALIENLASER:
                game.alien_shoot()  # Llama al método de disparo de los aliens
        
        action = random.choice([0,1,2,3])  # Selecciona una acción aleatoria
        state = game.step(action)  # Ejecuta el paso y recibe el estado

        # Muestra el puntaje y vidas
        #print(f"Score: {state['score']}, Lives: {state['lives']}")

        # Termina si el jugador pierde todas las vidas
        if state['lives'] <= 0:
            print("Game Over")
            break

        # Actualizar la pantalla
        game.screen.fill((30, 30, 30))  # Limpiar la pantalla con un color de fondo
        game.run()  # Ejecutar la lógica del juego (actualiza y dibuja)

        pygame.display.flip()  # Actualiza la pantalla
        clock.tick(60)  # 60 FPS


    pygame.quit()

except SystemExit:
        # Manejar SystemExit para evitar que el kernel se cierre
        print("El juego ha terminado de forma segura.")
        print(f"Tu puntaje fue: {state['score']}")

El juego ha terminado de forma segura.
Tu puntaje fue: 5500


In [9]:
import pygame
from main import Game
from agent import Agent  # Asegúrate de que Agent esté importado correctamente

try:
    # Inicializar Pygame y el juego
    pygame.init()
    screen_width = 800
    screen_height = 600
    screen = pygame.display.set_mode((screen_width, screen_height))
    clock = pygame.time.Clock()
    game = Game(screen_width, screen_height)
    agent = Agent(action_space=4, state_shape=(screen_height, screen_width, 3))  # Crear instancia del agente

    # Evento para el disparo de los aliens
    ALIENLASER = pygame.USEREVENT + 1
    pygame.time.set_timer(ALIENLASER, 800)  # Configura el temporizador para disparar cada 800 ms

    # Bucle de juego controlado por el agente
    for _ in range(10000):  # Ejecuta hasta 10000 pasos o hasta que termine
        for event in pygame.event.get():
            if event.type == ALIENLASER:
                game.alien_shoot()  # Llama al método de disparo de los aliens

        # Obtener la acción del agente
        action = agent.select_action(state)  # El agente selecciona una acción basada en el estado actual

        # Ejecutar la acción en el juego
        state = game.step(action)  # Ejecuta el paso y recibe el estado actualizado

        # Termina si el jugador pierde todas las vidas
        if state['lives'] <= 0:
            print("Game Over")
            break

        # Actualizar la pantalla
        game.screen.fill((30, 30, 30))  # Limpiar la pantalla con un color de fondo
        game.run()  # Ejecutar la lógica del juego (actualiza y dibuja)

        pygame.display.flip()  # Actualiza la pantalla
        clock.tick(60)  # 60 FPS

    # Imprimir resultado final
    print("El juego ha terminado de forma segura.")
    print(f"Tu puntaje fue: {state['score']}")

    pygame.quit()

except SystemExit:
    # Manejar SystemExit para evitar que el kernel se cierre
    print("El juego ha terminado de forma segura.")
    print(f"Tu puntaje fue: {state['score']}")

El juego ha terminado de forma segura.
Tu puntaje fue: 6400


In [ ]:
import pygame
import os
import numpy as np
from main import Game
from agent import Agent
import torch

# Configuración inicial de Pygame
pygame.init()
screen_width = 800
screen_height = 600
screen = pygame.display.set_mode((screen_width, screen_height))
clock = pygame.time.Clock()

# Configura el número de episodios y otros parámetros
num_episodes = 1000  # Número de episodios que deseas ejecutar
agent = Agent(action_space=4, state_shape=(screen_height, screen_width, 3))

# Almacena los resultados y modelos
results = []
model_dir = "models"  # Directorio para guardar modelos
os.makedirs(model_dir, exist_ok=True)  # Crea el directorio si no existe

# Intentar cargar un modelo existente
model_path = os.path.join(model_dir, "agent_model_latest.pt")
if os.path.exists(model_path):
    agent.model.load_state_dict(torch.load(model_path))
    print("Modelo cargado.")

for episode in range(num_episodes):
    game = Game(screen_width, screen_height)  # Reinicia el juego para un nuevo episodio
    done = False
    total_reward = 0

    while not done:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                done = True  # Permitir salir del juego si se cierra la ventana

        # Obtener el estado actual del juego
        state = game.get_state_image()
        action = agent.select_action(state)  # Elige una acción usando el agente

        # Toma una acción en el juego
        next_state, reward, done= game.step(action)

        # Almacena la transición en la memoria del agente
        agent.store_transition((state, action, reward, next_state, done))

        # Entrena al agente
        agent.train_step()  # Entrena al agente

        total_reward += reward

    results.append(total_reward)  # Almacena el resultado del episodio
    print(f"Episode {episode + 1}, Total Reward: {total_reward}")

    # Guarda el modelo después de cada episodio
    torch.save(agent.model.state_dict(), model_path)

# Guarda los resultados en un archivo
np.savetxt("results.txt", results, fmt="%d")

print("El entrenamiento ha finalizado de forma segura.")

# Asegurarse de que Pygame se cierre correctamente
pygame.quit()

TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint64, uint32, uint16, uint8, and bool.

: 

In [ ]:
import numpy as np
from collections import deque
from main import Game  # Asegúrate de que main.py contiene la clase Game
from dqn_model import DQN  # Asegúrate de que dqn_model.py contiene la clase DQN
import pygame

# Parámetros de entrenamiento
LEARNING_RATE = 1e-4
LEARNING_RATE_DECAY = 0.99
EXPLORATION_DECAY = 0.95
GAMMA = 0.975
UPDATE_TARGET_EVERY = 10
BATCH_SIZE = 32
EPISODES = 101

pygame.init()
# Inicialización del entorno y del agente
env = Game(screen_width=800, screen_height=600)  # Ajusta el tamaño según tu diseño
agent = DQN(
    state_shape=(800,600,3),  # Ajusta según la forma de tu imagen de estado
    action_size=4,  # Número de acciones en el juego (izquierda, derecha, disparar, esperar, por ejemplo)
    batch_size=BATCH_SIZE,
    learning_rate_max=LEARNING_RATE,
    learning_rate_decay=LEARNING_RATE_DECAY,
    exploration_decay=EXPLORATION_DECAY,
    gamma=GAMMA
)

# Inicializar el estado del entorno
state = env.get_state_image()
state = np.expand_dims(state, axis=0)  # Añadir dimensión para batch
most_recent_losses = deque(maxlen=BATCH_SIZE)
log = []

# Llenar la memoria antes de comenzar el entrenamiento
while agent.memory.length() < BATCH_SIZE:
    action = agent.act(state)  # Elegir una acción
    next_state, reward, done = env.step(action)  # Ejecutar la acción en el juego

    # Asegúrate de que el estado y el siguiente estado no tengan dimensiones extra
    next_state = next_state  # Extraer la imagen, si es necesario
    state_image = state  # Extraer la imagen actual
    agent.remember(state_image, action, reward, next_state, done)  # Guardar la experiencia
    state = np.expand_dims(next_state, axis=0)  # Preparar el nuevo estado

# Loop de entrenamiento
for e in range(EPISODES):
    state = env.get_state_image()  # Reiniciar el juego
    state = np.expand_dims(state, axis=0)  # Añadir dimensión
    done = False
    step = 0
    ma_loss = None

    while not done:
        action = agent.act(state)  # Elegir una acción
        next_state, reward, done = env.step(action)  # Ejecutar la acción
        next_state = np.expand_dims(next_state, axis=0)  # Preparar el nuevo estado
        agent.remember(state, action, reward, next_state, done)  # Guardar experiencia
        state = next_state  # Actualizar estado
        step += 1

        # Entrenamiento del agente
        loss = agent.replay(episode=e)  # Actualiza el agente con la experiencia almacenada
        if loss is not None:
            most_recent_losses.append(loss)
            ma_loss = np.mean(most_recent_losses)
            print(f"Step: {step}. -- Loss: {loss:.4f}", end="          \r")

        if done:
            print(f"Episode {e}/{EPISODES-1} completed with {step} steps. Score: {env.score:.0f}.")
            break

    # Guardar log del episodio
    log.append([e, step, env.score, ma_loss])

    # Guardar el modelo al completar cada episodio
    agent.save(f'models/episode_{e}.h5')

pygame 2.6.1 (SDL 2.28.4, Python 3.11.9)
Hello from the pygame community. https://www.pygame.org/contribute.html


c:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


KeyboardInterrupt: 

In [1]:
import numpy as np
from collections import deque
from main import Game  # Asegúrate de que main.py contiene la clase Game
from dqn_model import DQN  # Asegúrate de que dqn_model.py contiene la clase DQN
import pygame

# Parámetros de entrenamiento
LEARNING_RATE = 1e-4
LEARNING_RATE_DECAY = 0.99
EXPLORATION_DECAY = 0.95
GAMMA = 0.975
UPDATE_TARGET_EVERY = 10
BATCH_SIZE = 10
EPISODES = 101
MAX_STEPS_PER_EPISODE = 10  # Limite de pasos por episodio

pygame.init()
# Inicialización del entorno y del agente
env = Game(screen_width=800, screen_height=600)  # Ajusta el tamaño según tu diseño
agent = DQN(
    state_shape=(800, 600, 3),  # Ajusta según la forma de tu imagen de estado
    action_size=4,  # Número de acciones en el juego (izquierda, derecha, disparar, esperar, por ejemplo)
    batch_size=BATCH_SIZE,
    learning_rate_max=LEARNING_RATE,
    learning_rate_decay=LEARNING_RATE_DECAY,
    exploration_decay=EXPLORATION_DECAY,
    gamma=GAMMA
)

# Inicializar el estado del entorno
state = env.get_state_image()
state = np.expand_dims(state, axis=0)  # Añadir dimensión para batch
most_recent_losses = deque(maxlen=BATCH_SIZE)
log = []

# Llenar la memoria antes de comenzar el entrenamiento
while agent.memory.length() < BATCH_SIZE:
    action = agent.act(state)  # Elegir una acción
    next_state, reward, done = env.step(action)  # Ejecutar la acción en el juego

    # Asegúrate de que el estado y el siguiente estado no tengan dimensiones extra
    next_state = next_state  # Extraer la imagen, si es necesario
    state_image = state  # Extraer la imagen actual
    agent.remember(state_image, action, reward, next_state, done)  # Guardar la experiencia
    state = np.expand_dims(next_state, axis=0)  # Preparar el nuevo estado

# Loop de entrenamiento
for e in range(EPISODES):
    state = env.get_state_image()  # Reiniciar el juego
    state = np.expand_dims(state, axis=0)  # Añadir dimensión
    done = False
    step = 0
    ma_loss = None

    while not done and step < MAX_STEPS_PER_EPISODE:  # Limitar el número de pasos
        action = agent.act(state)  # Elegir una acción
        next_state, reward, done = env.step(action)  # Ejecutar la acción
        next_state = np.expand_dims(next_state, axis=0)  # Preparar el nuevo estado
        agent.remember(state, action, reward, next_state, done)  # Guardar experiencia
        state = next_state  # Actualizar estado
        step += 1

        # Entrenamiento del agente
        loss = agent.replay(episode=e)  # Actualiza el agente con la experiencia almacenada
        if loss is not None:
            most_recent_losses.append(loss)
            ma_loss = np.mean(most_recent_losses)
            print(f"Step: {step}. -- Loss: {loss:.4f}", end="          \r")

        if done:
            print(f"Episode {e}/{EPISODES-1} completed with {step} steps. Score: {env.score:.0f}.")
            break

    # Si el episodio termina debido a que se alcanzó el límite de pasos
    if step >= MAX_STEPS_PER_EPISODE:
        print(f"Episode {e}/{EPISODES-1} reached maximum steps with {step} steps. Score: {env.score:.0f}.")

    # Guardar log del episodio
    log.append([e, step, env.score, ma_loss])

    # Guardar el modelo al completar cada episodio
    agent.save(f'models/episode_{e}.keras')

pygame 2.6.1 (SDL 2.28.4, Python 3.11.9)
Hello from the pygame community. https://www.pygame.org/contribute.html


c:\Users\Admin\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Episode 0/100 reached maximum steps with 10 steps. Score: 0.
Episode 1/100 reached maximum steps with 10 steps. Score: 100.


KeyboardInterrupt: 

In [ ]:
import numpy as np
import pygame
from main import Game  # Asegúrate de que el nombre de tu archivo sea correcto
from dqn_model import DQN

# Carga del modelo entrenado
model_path = "models\episode_1.keras"
agent = DQN(state_shape=(800,600,3), action_size=4)  # Ajusta el tamaño de estado y acciones según tu implementación
agent.load(model_path)

# Inicialización del juego
pygame.init()
screen_width = 800  # Cambia según tu configuración
screen_height = 600  # Cambia según tu configuración
game_instance = Game(screen_width, screen_height)

clock = pygame.time.Clock()
running = True
state = game_instance.get_state_image()  # Asegúrate de que el método esté presente en tu clase Game

while running:
    pygame.display.set_caption(f"Score: {game_instance.score}")  # Muestra la puntuación actual

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    action = agent.act(state, 0)  # Actúa según el estado actual
    state, reward, done = game_instance.step(action)  # Realiza un paso en el juego

    # Actualiza el estado del juego
    if done:
        print("Game Over!")  # O maneja la finalización del juego de otra manera

    # Renderiza el estado en pantalla
    game_instance.render_game()  # Asegúrate de tener un método render que dibuje el estado del juego
    pygame.display.flip()
    clock.tick(30)

pygame.quit()

In [11]:
import random
import pygame
from main import Game 
import sys
import numpy as np
import pygame
from main import Game  # Asegúrate de que el nombre de tu archivo sea correcto
from dqn_model import DQN

# Carga del modelo entrenado
model_path = "models\episode_1.keras"
agent = DQN(state_shape=(800,600,3), action_size=4)  # Ajusta el tamaño de estado y acciones según tu implementación
agent.load(model_path)


try:
    # Inicializar Pygame y el juego
    pygame.init()
    screen_width = 800
    screen_height = 600
    screen = pygame.display.set_mode((screen_width, screen_height))
    game = Game(screen_width, screen_height)
    clock = pygame.time.Clock()
    state = game.get_state_image()

    # Evento para el disparo de los aliens
    ALIENLASER = pygame.USEREVENT + 1
    pygame.time.set_timer(ALIENLASER, 600)  # Configura el temporizador para disparar cada 800 ms

    # Bucle de juego aleatorio
    for _ in range(10000):  # Ejecuta 100 pasos, ajusta según sea necesario
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            if event.type == ALIENLASER:
                game.alien_shoot()
        
        action = agent.act(state, 0)
        state, reward, done = game.step(action)  # Ejecuta el paso y recibe el estado

        # Muestra el puntaje y vidas
        #print(f"Score: {state['score']}, Lives: {state['lives']}")

        # Termina si el jugador pierde todas las vidas
        if done:
            print("Game Over!")  # O maneja la finalización del juego de otra manera

        # Actualizar la pantalla
        game.screen.fill((30, 30, 30))  # Limpiar la pantalla con un color de fondo
        game.run()  # Ejecutar la lógica del juego (actualiza y dibuja)

        pygame.display.flip()  # Actualiza la pantalla
        clock.tick(60)  # 60 FPS


    pygame.quit()

except SystemExit:
        # Manejar SystemExit para evitar que el kernel se cierre
        print("El juego ha terminado de forma segura.")
        print(f"Tu puntaje fue: {game.score}")

El juego ha terminado de forma segura.
Tu puntaje fue: 1700
